# Samples: General

Sets up a simple APIM instance with a variety of policies to experiment.

⚙️ **Supported infrastructures**: All infrastructures

⌚ **Expected *Run All* runtime (excl. infrastructure prerequisite): ~1 minute**

## 🎯 Objectives

1. Experience a variety of policies in any of the infrastructure architectures. You may see several examples from our [APIM policy snippets repo](https://github.com/Azure/api-management-policy-snippets).
1. Become proficient with how policies operate.
1. Gain confidence in setting up and configuring policies appropriately.

## ⚙️ Configuration

1. Decide which of the [Infrastructure Architectures](../../README.md#infrastructure-architectures) you wish to use.
    1. If the infrastructure _does not_ yet exist, navigate to the desired [infrastructure](../../infrastructure/) folder and follow its README.md.
    1. If the infrastructure _does_ exist, adjust the `user-defined parameters` in the _Initialize notebook variables_ below. Please ensure that all parameters match your infrastructure.

### Initialize notebook variables

Configures everything that's needed for deployment. **Modify entries under _1) User-defined parameters_ and _3) Define the APIs and their operations and policies_**.

In [ ]:
import utils
from apimtypes import *

# 1) User-defined parameters (change these as needed)
rg_location = 'eastus2'
index       = 1
deployment  = INFRASTRUCTURE.SIMPLE_APIM
tags        = ['general']

# 2) Service-defined parameters (please do not change these)
rg_name = utils.get_infra_rg_name(deployment, index)
supported_infrastructures = [INFRASTRUCTURE.AFD_APIM_PE, INFRASTRUCTURE.APIM_ACA, INFRASTRUCTURE.SIMPLE_APIM]
utils.validate_infrastructure(deployment, supported_infrastructures)

# 3) Define the APIs and their operations and policies

# API 1
api1_get = GET_APIOperation('This is a GET for API 1')
api1_post = POST_APIOperation('This is a POST for API 1')
api1 = API('API1', 'API 1', '/api1', 'This is API 1', operations = [api1_get, api1_post], tags = tags)

# API 2
api2_post = POST_APIOperation('This is a POST for API 2')
api2 = API('API2', 'API 2', '/api2', 'This is API 2', operations = [api2_post], tags = tags)

# Request Headers
rh_policy_xml = utils.read_policy_xml(REQUEST_HEADERS_XML_POLICY_PATH)
rh_get = GET_APIOperation('Gets the request headers for the current request and returns them. Great for troubleshooting.', rh_policy_xml)
rh = API('requestheaders', 'Request Headers', '/request-headers', 'API for request headers', operations = [rh_get], tags = tags)

# APIs Array
apis: List[API] = [api1, api2, rh]

utils.print_ok('Notebook initialized')

### Create deployment using Bicep

Creates the bicep deployment into the previously-specified resource group. A bicep parameters file will be created prior to execution.

In [ ]:
import utils

# 1) Define the Bicep parameters with serialized APIs
bicep_parameters = {
    'apis': {'value': [api.to_dict() for api in apis]}
}

# 2) Infrastructure must be in place before samples can be layered on top
if not utils.does_resource_group_exist(rg_name):
    utils.print_error(f'The specified infrastructure resource group and its resources must exist first. Please check that the user-defined parameters above are correctly referencing an existing infrastructure. If it does not yet exist, run the desired infrastructure in the /infra/ folder first.')
    raise SystemExit(1)

# 3) Run the deployment
output = utils.create_bicep_deployment_group(rg_name, rg_location, deployment, bicep_parameters)

# 4) Print a deployment summary, if successful; otherwise, exit with an error
if not output.success:
    raise SystemExit('Deployment failed')

if output.success and output.json_data:
    apim_gateway_url = output.get('apimResourceGatewayURL', 'APIM API Gateway URL')

utils.print_ok('Deployment completed')

### Verify API Request Success

Assert that the deployment was successful by making simple calls to APIM. 

❗️ If the infrastructure shields APIM and requires a different ingress (e.g. Azure Front Door), the request to the APIM gateway URl will fail by design. Obtain the Front Door endpoint hostname and try that instead.

In [ ]:
import utils
from apimrequests import ApimRequests

# 1) Issue a direct request to API Management
reqsApim = ApimRequests(apim_gateway_url)
reqsApim.singleGet('/request-headers', msg = 'Calling Request Headers API via API Management Gateway URL. Response codes 200 and 403 are both valid depending on the infrastructure used.')

# 2) Issue requests against Front Door.
# Check if the infrastructure architecture deployment uses Azure Front Door.
utils.print_message('Checking if the infrastructure architecture deployment uses Azure Front Door.', blank_above = True)
afd_endpoint_url = utils.get_frontdoor_url(deployment, rg_name)

if afd_endpoint_url:
    reqsAfd = ApimRequests(afd_endpoint_url)
    reqsAfd.singleGet('/request-headers', msg = 'Calling Request Headers API via via Azure Front Door. Expect 200.')

utils.print_ok('All done!')